In [4]:
import pyspark.sql.functions as F
from credentials import MY_CREDENTIALS
from data_location import DELTA_LOCATION

from spark_bi.spark import FutPathlingContext

pc = FutPathlingContext.create(
    app_name="example-spark-app", hadoop_config=MY_CREDENTIALS.to_hadoop_config()
)
delta_lake = pc.read.delta(DELTA_LOCATION)

# By aktiv/inaktiv

Definitionen af, om en patient er "aktiv", kan være meget forskellig alt efter forretningsbehov.

I FHIR data har vi markeret en patient som aktiv, hvis de har mindst ét "åbent" forløb. Et "åbent" forløb defineres som:
* PLANNED
* WAITLIST
* ACTIVE
* ON_HOLD

Denne status ligger på `patient.active` og synkroniseres dagligt.

Hvis vi genbruger denne definition, kan vi hurtigt finde antal:

In [ ]:
(
    delta_lake.view(resource="Patient", select=[{"column": [{"name": "active", "path": "active"}]}])
    .groupby("active")
    .count()
    .toPandas()  # Pandas dataframes are rendered nicely in Jupyter Notebooks
)

,active,count
0,True,3
1,False,712


# By anvenderløsning

For at finde anvenderløsning bruger vi coexistence-tags der er beskrevet i: https://ehealth-dk.atlassian.net/wiki/spaces/EDTW/pages/2355986433/Multitenancy

Patienterne selv stammer fra CPR-registret, og har derfor ikke et coexistence-tag. Alle relevante patienter har mindst ét forløb, og derfor tæller vi antal unikke patienter fra episodes of care, og finder disses creating_solution.

In [ ]:
from pyspark.sql.functions import col

eoc_with_pt_id = delta_lake.view(
    resource="EpisodeOfCare",
    select=[
        {
            "column": [
                {"name": "eoc_id", "path": "getResourceKey()"},
                {"name": "eoc_patient_id", "path": "patient.getReferenceKey()"},
                {
                    "name": "creating_solution",
                    "path": "meta.tag.where(system='http://ehealth.sundhed.dk/cs/ehealth-system').code.first()",
                },
            ]
        },
        {"forEach": "team", "column": [{"name": "eoc_team_id", "path": "getReferenceKey()"}]},
    ],
)
eoc_with_pt_id.filter(col("creating_solution").isNotNull()).head(5)

[Row(eoc_id='EpisodeOfCare/2000469910', eoc_patient_id='Patient/1000684720', creating_solution='xb', eoc_team_id='CareTeam/3000143203'),
 Row(eoc_id='EpisodeOfCare/2000469920', eoc_patient_id='Patient/1000684720', creating_solution='xb', eoc_team_id='CareTeam/3000143203'),
 Row(eoc_id='EpisodeOfCare/2000469925', eoc_patient_id='Patient/1000684720', creating_solution='xb', eoc_team_id='CareTeam/3000143203'),
 Row(eoc_id='EpisodeOfCare/2000469931', eoc_patient_id='Patient/1000684720', creating_solution='xb', eoc_team_id='CareTeam/3000143203'),
 Row(eoc_id='EpisodeOfCare/2000469943', eoc_patient_id='Patient/1000684720', creating_solution='xb', eoc_team_id='CareTeam/3000143203')]

In [6]:
(
    eoc_with_pt_id.groupby("creating_solution")
    .agg(F.countDistinct("eoc_patient_id").alias("citizen_count"))
    .sort(F.desc("citizen_count"))
    .toPandas()
)

,creating_solution,citizen_count
0,None,3
1,xb,1


Bemærk at antallet er for TRIFORKs testmiljø, og derfor ikke retvisende.

# By careteam

Fordi hver episode of care kun kan være tilknyttet én patient, så kan vi tælle antal patienter per careteam ved at:
* Finde alle episodes of care knyttet til det careteam
* Tælle antal unikke patient-id'er blandt alle disse episodes of care

Det gør vi:

In [7]:
careteams = delta_lake.view(
    resource="CareTeam", select=[{"column": [{"name": "careteam_id", "path": "getResourceKey()"}]}]
)
careteams.head(5)

[Row(careteam_id='CareTeam/3000000001'),
 Row(careteam_id='CareTeam/3000000002'),
 Row(careteam_id='CareTeam/3000000005'),
 Row(careteam_id='CareTeam/3000000006'),
 Row(careteam_id='CareTeam/3000000007')]

In [8]:
eoc_with_pt_id = delta_lake.view(
    resource="EpisodeOfCare",
    select=[
        {
            "column": [
                {"name": "episodeofcare_id", "path": "getResourceKey()"},
                {"name": "patient_id", "path": "patient.getReferenceKey()"},
                {
                    "name": "organization_id",
                    "path": "managingOrganization.first().getReferenceKey()",
                },
            ]
        },
        {"forEach": "team", "column": [{"name": "team_id", "path": "getReferenceKey()"}]},
    ],
)
eoc_with_pt_id.head(5)

[Row(episodeofcare_id='EpisodeOfCare/2000000029', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000035', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000042', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000049', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752'),
 Row(episodeofcare_id='EpisodeOfCare/2000000068', patient_id='Patient/1000264558', organization_id='Organization/3000008564', team_id='CareTeam/3000108752')]

In [9]:
(
    careteams.join(eoc_with_pt_id, careteams.careteam_id == eoc_with_pt_id.team_id, how="left")
    .select("team_id", "patient_id")
    .distinct()
    .groupBy("team_id")
    .agg(F.count("*").alias("n_patients"))
    .sort("n_patients", ascending=False)
).head(5)

[Row(team_id='CareTeam/3000143203', n_patients=3),
 Row(team_id='CareTeam/3000000002', n_patients=2),
 Row(team_id='CareTeam/3000178177', n_patients=1),
 Row(team_id=None, n_patients=1)]

# By diagnosis

For at finde patientens diagnoser/behandlingsområder skal vi lave koblingen:

`Patient <-> EpisodeOfCare <-> CarePlan.addresses <-> Condition.code`

Vi starter fra højre:

In [10]:
conditions_with_code = delta_lake.view(
    resource="Condition",
    select=[
        {
            "column": [
                {"name": "condition_id", "path": "getResourceKey()"},
                {"name": "diagnosis_code", "path": "code.coding.code"},
            ]
        }
    ],
)
conditions_with_code.head(5)

[Row(condition_id='Condition/2000000003', diagnosis_code='DJ44'),
 Row(condition_id='Condition/2000000008', diagnosis_code='DJ44'),
 Row(condition_id='Condition/2000000036', diagnosis_code='DJ44'),
 Row(condition_id='Condition/2000000050', diagnosis_code='DJ44'),
 Row(condition_id='Condition/2000000069', diagnosis_code='DJ44')]

In [11]:
careplans_with_eoc = delta_lake.view(
    resource="CarePlan",
    select=[
        {
            "column": [
                {"name": "cp_id", "path": "getResourceKey()"},
                {
                    "name": "cp_eoc_id",
                    "path": "extension.where(url='http://hl7.org/fhir/StructureDefinition/workflow-episodeOfCare').valueReference.getReferenceKey()",
                },
                {"name": "addresses_condition", "path": "addresses.getReferenceKey()"},
            ]
        }
    ],
)
careplans_with_eoc.head(5)

[Row(cp_id='CarePlan/2000000027', cp_eoc_id='EpisodeOfCare/2000000023', addresses_condition='Condition/2000000024'),
 Row(cp_id='CarePlan/2000000039', cp_eoc_id='EpisodeOfCare/2000000035', addresses_condition='Condition/2000000036'),
 Row(cp_id='CarePlan/2000000072', cp_eoc_id='EpisodeOfCare/2000000068', addresses_condition='Condition/2000000069'),
 Row(cp_id='CarePlan/2000000084', cp_eoc_id='EpisodeOfCare/2000000080', addresses_condition='Condition/2000000081'),
 Row(cp_id='CarePlan/2000000115', cp_eoc_id='EpisodeOfCare/2000000111', addresses_condition='Condition/2000000112')]

In [12]:
eoc_with_pt_id = delta_lake.view(
    resource="EpisodeOfCare",
    select=[
        {
            "column": [
                {"name": "eoc_id", "path": "getResourceKey()"},
                {"name": "eoc_patient_id", "path": "patient.getReferenceKey()"},
            ]
        }
    ],
)
eoc_with_pt_id.head(5)

[Row(eoc_id='EpisodeOfCare/2000000029', eoc_patient_id='Patient/1000264558'),
 Row(eoc_id='EpisodeOfCare/2000000035', eoc_patient_id='Patient/1000264558'),
 Row(eoc_id='EpisodeOfCare/2000000042', eoc_patient_id='Patient/1000264558'),
 Row(eoc_id='EpisodeOfCare/2000000049', eoc_patient_id='Patient/1000264558'),
 Row(eoc_id='EpisodeOfCare/2000000068', eoc_patient_id='Patient/1000264558')]

Dernæst kan vi samle til én stor tabel:

In [13]:
(
    conditions_with_code.join(
        careplans_with_eoc,
        conditions_with_code["condition_id"] == careplans_with_eoc["addresses_condition"],
        how="inner",
    )
    .join(eoc_with_pt_id, careplans_with_eoc["cp_eoc_id"] == eoc_with_pt_id["eoc_id"], how="inner")
    .groupby("diagnosis_code")
    .agg(F.countDistinct("eoc_patient_id").alias("n_citizens"))
    .head(5)
)

[Row(diagnosis_code='DJ44', n_citizens=2)]

Vær opmærksom på, at disse tal er fra TRIFORKs testmiljø, og derfor ikke repræsentative.

# By kommune

Antal borgere per kommune fortolkes som:
* Antal borgere der er tilknyttet en episodeofcare, der er administreret af et careteam, der er administreret af kommunen

Derfor skal vi lave koblingen `EpisodeOfCare.team <-> CareTeam.managingOrganization <-> Org.municipalityCode`

In [14]:
from pyspark.sql.functions import col

eoc_municipality = delta_lake.view(
    resource="EpisodeOfCare",
    select=[
        {
            "column": [
                {"name": "eoc_id", "path": "getResourceKey()"},
                {"name": "eoc_patient_id", "path": "patient.getReferenceKey()"},
            ]
        },
        {"forEach": "team", "column": [{"name": "eoc_team_id", "path": "getReferenceKey()"}]},
    ],
)
eoc_municipality.head(5)

[Row(eoc_id='EpisodeOfCare/2000000029', eoc_patient_id='Patient/1000264558', eoc_team_id='CareTeam/3000108752'),
 Row(eoc_id='EpisodeOfCare/2000000035', eoc_patient_id='Patient/1000264558', eoc_team_id='CareTeam/3000108752'),
 Row(eoc_id='EpisodeOfCare/2000000042', eoc_patient_id='Patient/1000264558', eoc_team_id='CareTeam/3000108752'),
 Row(eoc_id='EpisodeOfCare/2000000049', eoc_patient_id='Patient/1000264558', eoc_team_id='CareTeam/3000108752'),
 Row(eoc_id='EpisodeOfCare/2000000068', eoc_patient_id='Patient/1000264558', eoc_team_id='CareTeam/3000108752')]

In [15]:
careteams_with_managing_org = delta_lake.view(
    resource="CareTeam",
    select=[
        {
            "column": [
                {"name": "ct_id", "path": "getResourceKey()"},
                {"name": "ct_org_id", "path": "managingOrganization.first().getReferenceKey()"},
            ]
        }
    ],
)
careteams_with_managing_org.filter(F.col("ct_org_id").isNotNull()).head(5)

[Row(ct_id='CareTeam/3000148060', ct_org_id='Organization/3000038806'),
 Row(ct_id='CareTeam/3000148061', ct_org_id='Organization/3000029719')]


Desværre er der på TRIFORK-miljøet kun 2 careteams der har en tilknyttet organisation. Vi fortsætter analysen.

In [16]:
organizations_with_municipality = delta_lake.view(
    resource="Organization",
    select=[
        {
            "column": [
                {"name": "org_id", "path": "getResourceKey()"},
                {
                    "name": "municipality_code",
                    "path": "extension('http://ehealth.sundhed.dk/fhir/StructureDefinition/ehealth-organization-municipalityCode').valueString",
                },
            ]
        }
    ],
)
organizations_with_municipality.head(5)

[Row(org_id='Organization/3000000064', municipality_code='0787'),
 Row(org_id='Organization/3000000069', municipality_code='0265'),
 Row(org_id='Organization/3000000072', municipality_code='0173'),
 Row(org_id='Organization/3000000088', municipality_code='0360'),
 Row(org_id='Organization/3000000090', municipality_code='0787')]

In [17]:
patients = delta_lake.view(
    resource="Patient", select=[{"column": [{"name": "patient_id", "path": "getResourceKey()"}]}]
)
patients.head(5)

[Row(patient_id='Patient/1000264558'),
 Row(patient_id='Patient/1000264559'),
 Row(patient_id='Patient/1000264560'),
 Row(patient_id='Patient/1000264604'),
 Row(patient_id='Patient/1000264605')]

In [18]:
joined = (
    eoc_municipality.join(
        careteams_with_managing_org,
        eoc_municipality.eoc_team_id == careteams_with_managing_org.ct_id,
        how="left",
    )
    .join(patients, eoc_municipality.eoc_patient_id == patients.patient_id, how="left")
    .join(
        organizations_with_municipality,
        careteams_with_managing_org.ct_org_id == organizations_with_municipality.org_id,
        how="left",
    )
    .filter(col("org_id").isNotNull())
)

joined.head(5)

[]

Det viser sig at der på TRIFORKs testmiljø ikke er nogle patienter med episodes of care for de careteams, der har `.managingOrganization`. 

## By bopælskommune

En alternativ fortolkning er antal borgere fordelt på bopælskommune:

In [19]:
patients_with_municipality = delta_lake.view(
    resource="Patient",
    select=[
        {
            "column": [
                {"name": "patient_id", "path": "getResourceKey()"},
                {
                    "name": "municipalityCode",
                    "path": "address.where(use = 'home').extension('http://hl7.dk/fhir/core/StructureDefinition/dk-core-municipalityCodes').valueCodeableConcept.coding.code",
                },
            ]
        }
    ],
)

(
    patients_with_municipality.groupBy("municipalityCode")
    .agg(F.countDistinct("patient_id").alias("n_patients"))
    .sort("n_patients", ascending=False)
    .toPandas()
)

,municipalityCode,n_patients
0,0575,18
1,0621,17
2,0370,16
3,0630,15
4,0756,15
...,...,...
93,0270,3
94,0849,2
95,0561,2
96,0840,2


# By organisatorisk enhed

Jeg er usikker på, om vi kan entydigt identificere organisatoriske enheder.

De organisatoriske enheder vil være repræsenterede som `ehealth-organization`, men en `ehealth-organization` er ikke opmærket med, hvilket niveau i SOR den afspejler.

En mulighed ville være at downloade hele SOR, finde kun de organisationer der har `SOR-type == "Organisatorisk enhed"`, og så bruge dette til at filtrere. Det udskydes for nuværende.

# By region

In [20]:
patients_with_region = delta_lake.view(
    resource="Patient",
    select=[
        {
            "column": [
                {"name": "patient_id", "path": "getResourceKey()"},
                {
                    "name": "regional_subdivision_code",
                    "path": "address.where(use = 'home').extension('http://hl7.dk/fhir/core/StructureDefinition/dk-core-RegionalSubDivisionCodes').valueCodeableConcept.coding.code",
                },
            ]
        }
    ],
)

# Map regional subdivision codes to region names
# See https://hl7.dk/fhir/core/1.1.0/ValueSet-dk-core-RegionalSubDivisionCodes.html
region_mapping = {
    "DK-81": "Nord Denmark Region",
    "DK-82": "Central Denmark Region",
    "DK-83": "Region of Southern Denmark",
    "DK-84": "Capital Region of Denmark",
    "DK-85": "Region Zealand",
}

# Apply mapping to the result
result_df = (
    patients_with_region.groupBy("regional_subdivision_code")
    .agg(F.countDistinct("patient_id").alias("n_patients"))
    .sort("n_patients", ascending=False)
    .toPandas()
)

result_df["region_name"] = result_df["regional_subdivision_code"].map(region_mapping)
result_df

,regional_subdivision_code,n_patients,region_name
0,DK-84,196,Capital Region of Denmark
1,DK-83,176,Region of Southern Denmark
2,DK-85,140,Region Zealand
3,DK-82,134,Central Denmark Region
4,DK-81,69,Nord Denmark Region
